In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Load the data
train_data_orig = pd.read_csv('train.csv')
test_data_orig = pd.read_csv('test.csv')

In [3]:
# # Create a dummy column for the 'Cabin' column in the test data
# test_data['Cabin'] = 'Unknown'

# combined_data = pd.concat([train_data_orig.drop('Transported', axis=1), test_data])

In [4]:
# Preprocessing functions
def process_cabin(df):
    df['Deck'] = df['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'Unknown')
    df['Side'] = df['Cabin'].apply(lambda x: x[-1] if pd.notnull(x) else 'Unknown')
    return df

def fill_cabin_nan(df):
    df.loc[(df['HomePlanet'] == 'Earth') & (df['Deck'] == 'Unknown'), 'Deck'] = 'E'
    df.loc[(df['HomePlanet'] == 'Mars') & (df['Deck'] == 'Unknown'), 'Deck'] = 'D'
    df.loc[(df['HomePlanet'] == 'Europa') & (df['Deck'] == 'Unknown'), 'Deck'] = 'B'
    return df

def fill_age_nan(df):
    children_age_limit = 12
    df.loc[(df['Age'].isnull()) & (df['HomePlanet'] == 'Mars'), 'Age'] = children_age_limit
    return df

def fill_side_nan(df):
    mode_values = df.groupby(['HomePlanet', 'Deck'])['Side'].agg(pd.Series.mode).to_dict()

    def fill_side(row):
        if pd.isna(row['Side']):
            key = (row['HomePlanet'], row['Deck'])
            if key in mode_values:
                return mode_values[key]
            else:
                return 'Unknown'
        else:
            return row['Side']

    df['Side'] = df.apply(fill_side, axis=1)
    return df

def fill_destination_nan(df):
    df.loc[(df['Age'] > 12) & (df['CryoSleep'] == 0) & (df['Destination'].isnull()), 'Destination'] = 'TRAPPIST-1e'
    return df

In [5]:
def preprocess_data(df):
    df = process_cabin(df)
    df = fill_cabin_nan(df)
    df = fill_age_nan(df)
    df = fill_side_nan(df)
    df = fill_destination_nan(df)

    # Check if the 'Transported' column exists before dropping it
    if 'Transported' in df.columns:
        df = df.drop(['Cabin', 'Name', 'PassengerId', 'Transported'], axis=1)
    else:
        df = df.drop(['Cabin', 'Name', 'PassengerId'], axis=1)

    # Encode categorical variables
    df = pd.get_dummies(df, columns=['HomePlanet', 'CryoSleep', 'Deck', 'Side', 'Destination', 'VIP'])

    # Fill NaNs in other columns with their medians
    df = df.fillna(df.median())

    return df

In [6]:
# Data preprocessing
train_data = preprocess_data(train_data_orig)
test_data = preprocess_data(test_data_orig)


In [7]:
# Separate the data
train_data_labels = train_data_orig['Transported']


In [8]:
# Train the model
X_train, X_val, y_train, y_val = train_test_split(train_data, train_data_labels, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
# Validate the model
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7797584818861415


In [10]:
# Predict on the test data
test_predictions = clf.predict(test_data)

In [11]:
# Create a submission file
submission_df = pd.DataFrame({
'PassengerId': test_data_orig['PassengerId'],
'Transported': test_predictions
})
submission_df.to_csv('submission.csv', index=False)